In [1]:
from transformers import pipeline

classifier = pipeline('sentiment-analysis')
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9598049521446228},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

토크나이저를 이용한 전처리  
- 입력을 토큰(token) 이라고 부르는 단어(word), 하위 단어(subword) 또는 기호(symbol)(예: 구두점)로 분할
- 각 토큰(token)을 정수(integer)로 매핑(mapping)
- 모델에 유용할 수 있는 부가적인 입력(additional inputs)을 추가
- 전처리는 모델이 사전 학습(pretraining)될 때와 정확히 동일한 방식으로 수행되어야 하므로 먼저 Model Hub에서 해당 정보를 다운로드해야 함. 이를 위해 AutoTokenizer 클래스와 from_pretrained() 메서드를 사용함
- 모델의 체크포인트(checkpoint) 이름을 사용하여 모델의 토크나이저(tokenizer)와 연결된 데이터를 자동으로 가져와 캐시함

In [2]:
from transformers import AutoTokenizer

checkpoint = 'distilbert-base-uncased-finetuned-sst-2-english'

# 이 tokenizer에 문장을 입력하면 모델에 전달할 수 있는 dictionary 정보를 구할수 있음
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [3]:
raw_inputs = ["I've been waiting for a HuggingFace course my whole life.",
              "I hate this so much!",]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors='pt')
print(inputs) # input_ids : 각 문장의 토큰 값

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [4]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertModel: ['classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model Heads
- 모델 헤드는 hidden states의 high-dimensional vector를 입력으로 받아 다른 차원에 투영함
- 일반적으로 head는 하나 또는 몇 개의 linear layers로 구성됨
- Transformer 모델의 출력은 처리할 모델 헤드(model head)로 직접 전달됨
- 모델은 임베딩 레이어(embeddings layer)와 후속 레이어(subsequent layers)로 표현됨
  - 임베딩 레이어(embeddings layer)는 토큰화된 입력(tokenized input)의 각 입력 ID를 해당 토큰을 나타내는 벡터(embeddings vector)로 변환함. 그 이후의 후속 레이어는 attention mechanism을 사용하여 이들 임베딩 벡터(embeddings vector)를 조작하여 문장의 최종 표현을 생성함

In [5]:
# AutoModel 클래스에서도 AutoModelForSequenceClassification 클래스를 사용하면
# 모델 헤드는 고차원 벡터를 입력으로 사용하고 2개의 값을 출력하게 됨

from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)
print(inputs)
print(outputs.logits)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}
tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)


출력 후처리  
- 모델은 첫 번째 문장에 대해 [-1.5607, 1.6123], 두 번째 문장에 대해 [4.1692, -3.3464]를 예측함
  - 이는 확률이 아니라 모델의 마지막 계층에서 출력된 정규화되지 않은 원시 점수인 logits 값임
  - 이들 값을 확률로 변환하려면 SoftMax 계층을 통과해야 함
  - 모든 Transformers 모델은 이 logits 값을 출력함. 그 이유는 일반적으로 학습을 위한 손실 함수는 최종 activation function와 실제 손실 함수를 모두 사용하여 구현되기 때문

In [6]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(model.config.id2label) # 모델의 레이블 값
print(predictions)

{0: 'NEGATIVE', 1: 'POSITIVE'}
tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)


AutoModel  
지정된 체크포인트를 바탕으로 생성된 모델의 wrapper class

In [7]:
# 설정 객체 로드
from transformers import BertConfig, BertModel

# config 생성
config = BertConfig()

# 해당 config에서 모델 생성
model = BertModel(config)

print(config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.28.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [8]:
# 이미 사전 학습된 모델 로드하는 것
from transformers import BertModel
model = BertModel.from_pretrained('bert-base-cased')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


- 모델의 아키텍처가 다르더라도 변경할 체크포인트가 현재 체크포인트와 유사한 task, 예를 들어, 감성 분석 태스크로 학습되었다면 변경이 가능해야 함
- bert-base-cased를 통해 pretrained model을 로드하면 BERT 개발자가 직접 학습한 모델 체크포인트를 로드할 수 있음
 - 이제 해당 모델은 체크포인트의 모든 가중치로 초기화됨
 - 학습된 task에 대한 inference에 사용할 수 있으며 새로운 task에 대해 fine-tune 수도 있음

In [9]:
# 모델 저장
# model.save_pretrained("saving_folder")

In [10]:
# 토크나이저는 input IDs 라는 어휘 인덱스로 변환함
sequences = ["Hello!", "Cool.", "Nice!"]

# 출력 값은 인코딩된 시퀀스의 이중 리스트
encoded_sequences = [
    [101, 7592, 999, 102], [101, 4658, 1012, 102], [101, 3835, 999, 102]]

import torch
model_inputs = torch.tensor(encoded_sequences)

# 모델의 입력으로 텐서 활용
# output = model(model_inputs)

토크나이저  
- 입력된 텍스트를 모델에서 처리할 수 있는 데이터로 변환하는 것
- 모델은 숫자만 처리할 수 있으므로 토크나이저는 텍스트 입력을 숫자 데이터로 변환해야 함
- 모델에 가장 적합하면서 최대한 간결한 표현을 찾는 것

단어 기반 토큰화 (Word-based Tokenization)  
- 예를 들어 아래 그림에서의 토큰화 과정은 원시 텍스트를 단어로 나누고 각각에 대한 숫자 표현을 찾는 것:
  - Let's | do | tokenization! -> 공백으로 분리
  - Let | 's | do | tokenization | ! | -> 구두점으로 분리

In [11]:
# 공백을 기준으로 토큰화
tokenized_text = "Jim Henson was a puppeteer".split()
print(tokenized_text)

['Jim', 'Henson', 'was', 'a', 'puppeteer']


- 어휘집(vocabulary)에 없는 단어를 표현하기 위해 사용자 정의 토큰이 필요함
  - "unknown" 토큰으로 알려져 있으며 종종 "[UNK]" 또는 ""로 표시됨
  - 토크나이저가 이러한 "unknown" 토큰을 많이 생성한다는 것은 토크나이저가 해당 단어의 합당한 표현(sensible representation)을 찾을 수 없고 그 과정에서 정보를 잃어버린다는 뜻
  - 어휘집(vocabulary)을 만들 때 토크나이저가 이러한 "unknown" 토큰들을 최대한 적게 출력하게끔 하는 것이 목표가 되어야 함

문자 기반 토큰화 (Character-based Tokenization)  
- 텍스트를 단어(words)가 아닌 문자(characters)로 나눔
- 장점
  - 어휘집(vocabulary)의 크기가 매우 작음
  - 모든 단어들이 문자를 가지고 만들어질 수 있기 때문에 out-of-vocabulary (OOV, unknown) 토큰이 적음
L | e | t | ' | s | d | o | t | o | k | e | n | i | z | a | t | i | o | n | ! |
- 위의 방식은 분리된 토큰 표현 자체가 단어가 아닌 문자 기반이므로 직관적으로 볼 때 각 토큰의 의미 파악이 어려움
- 그러나 이 또한 언어에 따라 다른데, 중국어에서 각 문자(한자)는 라틴(Latin) 언어의 문자보다 더 많은 정보를 전달함
- 고려해야 할 또 다른 사항은 모델에서 처리할 매우 많은 양의 토큰이 발생하게 된다는 것
  - 단어기반 토크나이저를 사용하면 각 단어는 하나의 단일 토큰이지만 문자로 변환하면 10개 이상의 토큰으로 쉽게 변환될 수 있음
- 위 두가지 방식의 장점을 최대한 활용하기 위해 이를 결합한 subword tokenization(하위단어 토큰화) 를 사용할 수 있음

하위 단어 토큰화 (Subword Tokenization)  
- 빈번하게 사용하는 단어(frequently used words)는 더 작은 하위단어(subword)로 분할하지 않고 희귀 단어(rare words)를 의미있는 하위 단어(meaningful subwords)로 분할해야 한다는 원칙에 기반함
  - 예를 들어 "annoyingly"는 희귀 단어로 간주될 수 있으며, "annoying"와 "ly"로 분해될 수 있음
  - 이들은 둘 다 독립적인 하위단어(standalone subwords)로 더 자주 출현할 가능성이 높으며 동시에 "annoyingly"의 의미는 "annoying"와 "ly"의 합성 의미(composite meaning)로 유지됨

다음은 하위 단어 토큰화(subword tokenization) 알고리즘이 "Let's do tokenization!" 시퀀스를 토큰화하는 방법을 보여주는 예    
Let's\</w> | do\</w> | token | ization\</w> | !\</w> |
  
- 위에서의 하위 단어들(subwords)은 충분한 양의 의미 정보(semantic meaning)을 제공하고 있dma
- 길이가 긴 한 단어를 표현하기 위해서 단 두 개의 토큰만 필요함. 이를 통해 구성 어휘가 많지 않은 어휘집(vocabulary)으로도 충분히 많은 수의 토큰들을 표현할 수 있고 "unknown" 토큰이 거의 없음
- 세부 기법들
  - Byte-level BPE (GPT-2에 사용됨)
  - WordPiece (BERT에 사용됨)
  - SentencePiece, Unigram (몇몇 다국어 모델에 사용됨)

In [12]:
from transformers import BertTokenizer, AutoTokenizer

# BERT와 동일한 체크포인트(checkpoint)로 학습된 BERT 토크나이저를 로드하는 것은 
# 모델을 로드하는 것과 동일한 방식으로 수행됨
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

# AutoTokenizer 클래스는 라이브러리에서 토크나이저 클래스를 가져옴
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
print(tokenizer("Using a Transformer network is simple"))

# 모델 저장
# tokenizer.save_pretrained("saving_folder")

{'input_ids': [101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}


인코딩 (Encoding)  
- 텍스트를 숫자로 변환하는 과정
- 토큰화 -> 입력 식별자(input IDs)로의 변환이라는 2단계 프로세스로 수행됨
- 첫 번째 단계는 텍스트를 토큰(tokens) 이라고 부르는 단어(또는 단어의 일부, 구두점 기호 등)로 분리하는 것
  - 본인이 사용하고자 하는 모델의 이름을 이용하여 토크나이저도 인스턴스화(instantiate)해야 함. 그래야 해당 모델에서 사용한 토크나이저를 동일하게 사용할 수 있음
- 두 번째 단계는 토큰화 결과인 토큰들을 숫자로 변환하여 텐서(tensor)를 만들고 이를 모델에 입력할 수 있도록 하는 것
  - 이를 위해 토크나이저는 from_pretrained() 메서드로 인스턴스화할 때 다운로드되는 파일 중의 하나로 vocabulary 를 포함하고 있음.
  - 모델이 사전학습될 때 사용한 것과 동일한 어휘집(vocabulary)을 사용해야 함

In [13]:
# 토큰화 작업
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
sequence = 'Using a Transformer network is simple'
tokens = tokenizer.tokenize(sequence)
print(tokens)

['Using', 'a', 'Trans', '##former', 'network', 'is', 'simple']


In [14]:
# 토큰을 입력 식별자로 변환 이후 각 토큰들의 입력 식별자(input IDs)로의 변환은 
# convert_tokens_to_ids() 메서드로 함
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[7993, 170, 13809, 23763, 2443, 1110, 3014]


In [15]:
# 디코딩 (Decoding)
# 디코딩(decoding)은 반대 방향으로 진행됨
# 변환된 입력 식별자(input IDs)를 이용해서 vocabulary에서 해당 문자열을 찾음
# decode() 메서드를 사용하여 수행함
decoded_string = tokenizer.decode([7993, 170, 13809, 23763, 2443, 1110, 3014])
print(decoded_string)

Using a Transformer network is simple


- decode() 메서드는 인덱스를 다시 토큰으로 변환할 뿐만 아니라 하위 단어(subword)로 분할된 토큰을 병합하여 원본 문장을 도출함
- 이 동작은 새로운 텍스트를 생성하는 모델인 프롬프트에서의 텍스트 생성, 번역, 요약 등과 같은 시퀀스-투-시퀀스(sequence-to-sequence) 문제 등을 다룰 때 사용함

다중 시퀀스 처리

In [16]:
# 모델(model)은 입력의 배치(batch) 형태를 요구함
# 숫자 리스트를 텐서(tensor)로 변환하고 모델에 입력
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = 'distilbert-base-uncased-finetuned-sst-2-english'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."
tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor(ids)

# 해당 코드는 런타임 에러 발생함
# model(input_ids)

- 위 문제는 모델에 하나의 단일 시퀀스를 입력해서 발생하는 문제
- Transformers 모델은 기본적으로 다중 문장 시퀀스를 한번에 입력하기를 기대한다는 것
- 아래 코드를 보면 입력 식별자(input IDs) 리스트를 텐서로 변환하는 동시에 차원 하나가 그 위에 추가되는 것을 알 수 있음

In [17]:
tokenized_inputs = tokenizer(sequence, return_tensors='pt')
print(tokenized_inputs['input_ids'])

tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102]])


In [18]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = 'distilbert-base-uncased-finetuned-sst-2-english'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor([ids])
print('Input IDs:', input_ids)

output = model(input_ids)
print('Logits:', output.logits)

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


In [19]:
# 위 코드에서는 입력 식별자(input IDs)와 그 결과 로짓(logit) 값을 출력하고 있음
# Batching은 모델을 통해 한번에 여러 문장을 입력하는 동작
# 문장이 하나만 있는 경우 아래와 같이 단일 시퀀스로 배치(batch)를 빌드할 수 있음
batch_ids = [ids, ids]

- 위 코드는 동일한 두 시퀀스로 구성된 배치임
- 배치 처리를 통해서 모델이 여러 문장을 동시에 입력받을 수 있도록 할 수 있음
- 다중 시퀀스를 사용하는 것은 두 개(또는 그 이상) 문장을 함께 배치 처리하려고 할 때 각 문장의 길이가 다를 수 있음
- 입력 식별자(input IDs) 리스트를 텐서로 직접 변환할 수 없고 일반적으로 padding을 채움

In [20]:
# 입력을 패딩(padding)하기
# 다음 리스트는 텐서로 변환할 수 없음
batched_ids = [[200, 200, 200], [200, 200],]

- 이 문제를 해결하기 위해 패딩(padding) 을 사용하여 텐서를 직사각형 모양으로 만듦
- 패딩은 길이가 더 짧은 문장에 패딩 토큰(padding token) 이라는 특수 단어를 추가하여 모든 문장이 동일한 길이를 갖도록 함
  - 10개의 단어로 구성된 10개의 문장과 20개의 단어가 있는 1개의 문장이 있는 경우 패딩(padding)을 사용하면 모든 문장에 20개의 단어가 포함됨

In [21]:
padding_id = 100
batched_ids = [[200, 200, 200], [200, 200, padding_id],]

# padding token의 id는 tokenizer.pad_token_id에 지정되어 있음
# 두 개의 시퀀스를 한번은 개별적으로 또 한번은 배치(batch) 형태로 모델에 입력할 경우
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [[200, 200, 200], [200, 200, tokenizer.pad_token_id],]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)


- 배치 처리된 예측 결과의 logits에 문제가 있음. 두 번째 행은 두 번째 문장의 logits과 같아야 하지만 완전히 다른 값을 갖음
- 이는 Transformer모델의 특징이 각 토큰을 컨텍스트화하는 attention layers를 가지고 있다는 것
- attention layer는 시퀀스의 모든 토큰에 주의 집중을 하기 때문에 패딩 토큰도 역시 고려 대상임
- 모델에 길이가 다른 개별 문장들을 입력할 때나 동일한 문장으로 구성된 패딩이 적용된 배치(batch)를 입력할 때 동일한 결과를 얻기 위해서는 패딩 토큰을 무시하도록 해야 함

어텐션 마스크 (attention masks)  
- attention mask는 0과 1로 채워진 입력 식별자(input IDs) 텐서(tensor)와 형태가 정확하게 동일한 텐서
- 1은 해당 토큰에 주의를 기울여야 함이고, 0은 해당 토큰을 무시해야 함
  - 모델의 attention layers에서 무시해야 함

In [22]:
batch_ids = [[200, 200, 200], [200, 200, tokenizer.pad_token_id]]
attention_mask = [[1, 1, 1], [1, 1, 0],]

outputs = model(
    torch.tensor(batch_ids), attention_mask=torch.tensor(attention_mask))

# batch의 두 번째 문장에 대해 동일한 로짓(logits) 값을 얻을 수 있음
print(outputs.logits)

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


In [23]:
# Transformer 모델을 사용할 때 모델에 입력할 수 있는 시퀀스의 길이에 제한이 있음
# max_sequence_length 매개변수 등을 지정하여 시퀀스를 절단하는 것이 좋음
max_sequence_length = 512
sequence = sequence[:max_sequence_length]